In [ ]:
!pip install -U transformers==3.0.0

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git

In [ ]:
%cd question_generation

In [35]:
from pipelines import pipeline

In [36]:
import re

In [37]:
nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [38]:
def clean_step(my_word):
    my_word = '    ' + my_word + '    '
    for i in range(len(my_word)):
        if my_word[i] == '[' and my_word[i+4] == ']':
            rmv = str(my_word[i+1]) + str(my_word[i+2]) + str(my_word[i+3])
            my_word = my_word.replace(rmv, ']]]')
        elif my_word[i] == '[' and my_word[i+3] == ']':
            rmv = str(my_word[i+1]) + str(my_word[i+2])
            my_word = my_word.replace(rmv, ']]')
        elif my_word[i] == '[' and my_word[i+2] == ']':
            rmv = str(my_word[i+1])
            my_word = my_word.replace(rmv, ']')
    return my_word

In [39]:
def clean_text(text):
    text = clean_step(text)
    text = text.encode("utf-8",errors='ignore').decode("utf-8")
    text = re.sub("https?:.*(?=\s)",'',text)
    text = re.sub("[‘’\"]","'",text)
    text = re.sub("[^\x00-\x7f]+",'',text)
    text = re.sub('\S*@\S*\s?',' ',text)
    text = re.sub('[#&\\*+/<>@[\]^`{|}~ \t\n\r]',' ',text)
    text = re.sub('\(.*?\)','',text)
    text = re.sub('\=\=.*?\=\=','',text)
    text = re.sub(' , ',',',text)
    text = re.sub(' \.','.',text)
    text = re.sub("  +",' ',text)
    text = re.sub(";",'and',text)
    return text.strip()

In [48]:
with open('/content/question_generation/egypt_sample.txt', encoding='utf-8') as f:
    test = f.read()

contexts = []
paragraphs = test.split('\n\n')

for i in paragraphs:
    if len(i) < 320 or i.count('\n') > 2:
        pass
    else:
        contexts.append(i)

In [49]:
for i in range(len(contexts)):
    contexts[i] = clean_text(contexts[i])

In [ ]:
nlps = []
semiFinalContexts = []
for i in range(len(contexts)):
  print('Paragraph:', i)
  try:
    nlps.append(nlp(contexts[i]))
    semiFinalContexts.append(contexts[i])
  except Exception:
    continue

In [54]:
finalContexts = []
questions = []
answers = []

In [55]:
for i in range(len(nlps)):
  for j in nlps[i]:
    finalContexts.append(contexts[i])
    questions.append(j['question'])
    answers.append(j['answer'])

In [ ]:
for i in range(len(questions)):
  print(finalContexts[i])
  print(questions[i])
  print(answers[i])

In [57]:
import pandas as pd

In [58]:
dataframe = {
    'Context': finalContexts,
    'Question': questions,
    'T5 Answer': answers
}

In [59]:
df = pd.DataFrame(dataframe)

In [62]:
df.head()

,Context,Question,T5 Answer
0,"Egypt,officially the Arab Republic of Egypt, i...",What land bridge forms the border between Afri...,Sinai Peninsula
1,"Egypt,officially the Arab Republic of Egypt, i...",What is Egypt bordered by to the north?,the Mediterranean Sea
2,"Egypt,officially the Arab Republic of Egypt, i...",What separates Egypt from Jordan and Saudi Ara...,Gulf of Aqaba
3,"Egypt,officially the Arab Republic of Egypt, i...",What is the second-largest city in Egypt?,Alexandria
4,"Egypt,officially the Arab Republic of Egypt, i...",How many people live in Egypt?,100 million


In [65]:
df.to_csv('/content/question_generation/QA_by_T5.csv')